In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import smallgraph as sg
import networkx as nx
from matplotlib import pyplot as plt
import lmz
import eden.graph as eg
import eden.display as ed
from yoda.graphs import ali2graph
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount

In [ ]:
from ubergauss import optimization as op
import yoda.ml.simpleMl as sml


def eval(ali,labels,
         ct = .95, 
         RYthresh = 0,
         norm = True, 
         d1 = .7,
         d2= .4,
         bad_weight = 0.1,
         fix_edges = True,
         # boost_range = 3, boost_thresh = .5, boost_weight = 1,
         min_r = 2, 
         nest = True,
         min_d = 1,**nothingtosee):
    
    graphs = lmz.Map(ali2graph.set_weight_label, ali, RYthresh=RYthresh)
    graphs = lmz.Map(ali2graph.dillute, graphs, dilute1 = d1, dilute2 = d2, fix_edges = fix_edges )
    graphs = lmz.Map(ali2graph.set_weight, graphs,  bad_weight = bad_weight,  consThresh= ct )
    
    if nest:
        graphs = lmz.Map(ali2graph.donest, graphs)
    # graphs = lmz.Map(ali2graph.nearstem, graphs, boost_range = boost_range, boost_thresh = boost_thresh, boost_weight = boost_weight)
    # graphs = [a.graph for a in ali]
    
    
    # matrix = vectorize_graphs_hack(graphs,disthack = kernelhack, normalization=norm, min_r = min_r,min_d=min_d,r=r,d=d, inner_normalization=norm) #.toarray() 
    matrix = eg.vectorize(graphs, normalization=norm,
                      min_r = min_r,min_d=min_d,
                      inner_normalization=norm) #.toarray()
        
    # ret= sml.knn_accuracy(matrix,labels)
    #ret = sml.knn_accuracy(matrix,labels)
    matrix = 1- (matrix @ matrix.T).toarray()
    ret = sml.average_precision(-matrix,labels)
    ret = {'score': ret} # , 'score_many': sml.knn_accuracy(matrix,labels, select_labels = manyseq)}
    return ret




space = '''min_r 0 5 1
min_d 0 5 1
nest 0 1 1
fix_edges 0 1 1
ct .6 1
d1 0 1
d2 0 1
bad_weight 0 1'''

#a,l, te = sg.makedata()
# eval(a,l,**{'min_d': 3, 'min_r': 4, 'nest': 1})


In [ ]:
# make some data
datasets = [   sg.makedata(splits = 3)[:2] for i in range (6)  ]

In [ ]:
space = sg.string_to_space(space)
parameters = [space.sample() for i in range(100)]

In [ ]:

results = [ op.gridsearch(eval, parameters , tasks = parameters , data= [a,l], mp=True)  for a,l in datasets]


In [ ]:
import pandas as pd
results_merged = pd.concat(results, ignore_index=False)

In [ ]:
result =  results_merged
# Calculate the average score for each index
average_scores = result.groupby(result.index)['score'].mean()

# Rank the average scores
ranks = average_scores.rank(method='min')

# Map the ranks back to the original DataFrame
result['average_score'] = result.index.map(average_scores)
result['rank'] = result.index.map(ranks)

In [ ]:
import seaborn as sns
sns.lineplot(data = result, x='rank', y= 'score')